# LeNet implementation

[Original video](https://youtu.be/fcOW-Zyb5Bo)

[LeNet paper](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf)

In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [9]:
# LeNet architecture
# 1x32x32 input -> k=5x5, s=1, p=0 -> avg pool, s=2, p=0 -> k=5x5, s=1, p=0 ->
# -> avg pool, s=2, p=0 -> conv 5x5 to 120 channels -> linear 84 -> linear 10
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2,2), stride=(2,2), padding=(0,0))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5),
                               stride=(1,1), padding=(0,0))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5),
                               stride=(1,1), padding=(0,0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5),
                               stride=(1,1), padding=(0,0))
        self.linear1 = nn.Linear(in_features=120, out_features=84)
        self.linear2 = nn.Linear(in_features=84, out_features=10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = x.reshape(x.shape[0], -1)  # (batch, 120, 1, 1) -> (batch, 120)

        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)

        return x

In [10]:
x = torch.randn(64, 1, 32, 32)
model = LeNet()
print(model(x).shape)

torch.Size([64, 10])


## Train the model

In [11]:
# Hyperparameters
in_channel = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 1024
num_epochs = 4  # 40

load_model = True
filename = 'my_checkpoint.pth.tar'

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set transforms
my_transforms = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
])

# For the error: HTTPError: HTTP Error 403: Forbidden
# StackOverflow: https://stackoverflow.com/a/66461122/7550928
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=my_transforms, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=my_transforms, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = LeNet().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)


def save_checkpoint(state, filename=filename):
    print(' => Saving checkpoint')
    torch.save(state, filename)


def load_checkpoint(checkpoint):
    print(' => Loading checkpoint')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_acc = checkpoint['acc']


if load_model and os.path.exists(filename):
    load_checkpoint(torch.load(filename))
else:
    best_acc = 0


# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data', end='')
    else:
        print('Checking accuracy on test data', end='')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = float(num_correct) / float(num_samples) * 100
        #print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

        model.train()
        return acc

 => Loading checkpoint


In [12]:
# Train network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()
    
    mean_loss = sum(losses)/len(losses)
    acc = check_accuracy(test_loader, model)
    msg = f'\rLoss at epoch {epoch} is {mean_loss:.5f}. Accuracy is {acc:.2f}'

    if best_acc < acc:
        best_acc = acc
        checkpoint = {'state_dict': model.state_dict(),
                      'optimizer': optimizer.state_dict(),
                      'acc': best_acc}
        print(msg, end='')
        save_checkpoint(checkpoint)
    else:
        print(msg)

Loss at epoch 0 is 0.23666. Accuracy is 93.41 => Saving checkpoint
Loss at epoch 1 is 0.22792. Accuracy is 93.47 => Saving checkpoint
Loss at epoch 2 is 0.22164. Accuracy is 93.82 => Saving checkpoint
Loss at epoch 3 is 0.21383. Accuracy is 93.85 => Saving checkpoint
Loss at epoch 4 is 0.20679. Accuracy is 94.23 => Saving checkpoint
Loss at epoch 5 is 0.20062. Accuracy is 94.38 => Saving checkpoint
Loss at epoch 6 is 0.19383. Accuracy is 94.65 => Saving checkpoint
Loss at epoch 7 is 0.18762. Accuracy is 94.74 => Saving checkpoint
Loss at epoch 8 is 0.18135. Accuracy is 94.95 => Saving checkpoint
Loss at epoch 9 is 0.17647. Accuracy is 95.06 => Saving checkpoint
Loss at epoch 10 is 0.17071. Accuracy is 95.11 => Saving checkpoint
Loss at epoch 11 is 0.16654. Accuracy is 95.39 => Saving checkpoint
Loss at epoch 12 is 0.16129. Accuracy is 95.42 => Saving checkpoint
Loss at epoch 13 is 0.15722. Accuracy is 95.63 => Saving checkpoint
Loss at epoch 14 is 0.15345. Accuracy is 95.63
Loss at epo

In [13]:
print(f': {check_accuracy(train_loader, model):.2f}')
print(f': {check_accuracy(test_loader, model):.2f}')

Checking accuracy on training data: 97.23
Checking accuracy on test data: 97.33
